In [ ]:
import pickle5 as pickle
with open('../../data/COVID-QA/top_N_ents_spacy-COVID_QA.pkl', 'rb') as f:
    top_N_ents = pickle.load(f)

In [ ]:
import wikipedia
from tqdm.auto import tqdm

search_res = {}

for i, ent in tqdm(enumerate(top_N_ents)):
    #Skipping those entities which don't return anything
    if wikipedia.search(ent) != []:
        search_res[ent] = wikipedia.search(str(ent), results=1)[0]

In [ ]:
import wikipedia
from transformers import AutoTokenizer, AutoModel
import torch

context_dict = {}

filtering = False
if filtering == True:
    filtering_threshold = 0.5
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    #We're consider scibert because pubmedbert assigns very high similarity for both related/unrelated terms.
    checkpoint = 'allenai/scibert_scivocab_uncased'
    
    model = AutoModel.from_pretrained(checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    
    cos = torch.nn.CosineSimilarity(dim=0)
    model.to(device)

for ent,res in tqdm(search_res.items()):
    if filtering == False:
        try:
            context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
        except:
            continue
    else:
        encoded_input = tokenizer([ent, res], return_tensors='pt', padding=True)
        with torch.no_grad():
            output = model(**encoded_input)
        
        similarity = cos(output.pooler_output[0], output.pooler_output[1])
        '''
        we're taking less than here since the similarity scores for related terms seem to be lower than 
        unrelated ones.
        '''
        if similarity.item() < filtering_thresold:
            try:
                context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
            except:
                continue

In [ ]:
import pandas as pd
pd.DataFrame(context_dict.items(), columns = ['ent', 'text']).to_parquet('wiki_corpus.parquet', index=False)

In [ ]:
from pymed import PubMed

# Create a PubMed object that GraphQL can use to query
# Note that the parameters are not required but kindly requested by PubMed Central
# https://www.ncbi.nlm.nih.gov/pmc/tools/developers/
pubmed = PubMed(tool="MyTool", email="my@email.address")

# Create a GraphQL query in plain text
query = ['covid-19', 'hiv-1']

# Execute the query against the API
results = pubmed.query(query, max_results=500)

# Loop over the retrieved articles
for article in results:

    # Extract and format information from the article
    article_id = article.pubmed_id
    title = article.title
    abstract = article.abstract

    # Show information about the article
    print(
        f'{abstract}\n'
    )

In [ ]:
from datasets import load_dataset

In [ ]:
from tqdm.auto import tqdm
import json
with open('../../data/policy-qa/train.json', 'r') as f:
    data = json.load(f)

In [ ]:
#Removing unnecessary columns to cast to SQuAD format.
for article in tqdm(data['data']):
    title = article.get("title", "")
    for paragraph in article["paragraphs"]:
        paragraph.pop('summary', None)
        paragraph.pop('index', None)
        for qa in paragraph["qas"]:
            qa.pop('type', None)

In [ ]:
from pprint import pprint
for article in data['data']:
    title = article.get("title", "")
    print(article.keys())
    for paragraph in article["paragraphs"]:
        print(paragraph.keys())
        for qa in paragraph["qas"]:
            print(qa.keys())
            break
        break
    break

In [ ]:
s = load_dataset('Saptarshi7/PolicyQA')

In [ ]:
for article in s['train']['data'][0]:
    print(title)
    for paragraph in article["paragraphs"]:
        context = paragraph["context"]  # do not strip leading blank spaces GH-2585
        print(paragraph['summary'])
        break
    break

In [ ]:
s['train']['data'][1]

In [41]:
from datasets import load_dataset
s = load_dataset('Saptarshi7/covid_qa_cleaned_CS')

Found cached dataset covid_qa_cleaned_cs (/home/saptarshi/.cache/huggingface/datasets/Saptarshi7___covid_qa_cleaned_cs/covid_qa_cleaned_CS/1.0.0/74f520624df6ee24fe0a236d66e7f206080b9649fed5d52129ca286f6dd40206)


  0%|          | 0/1 [00:00<?, ?it/s]

In [40]:
len(set(s['test']['document_id']).intersection(set(s['train']['document_id'])))

98

In [155]:
len(test['document_id'])

343

In [116]:
train = s['train'].select(range(0, 1676))
train['document_id'][-1]

2504

In [165]:
train

Dataset({
    features: ['document_id', 'context', 'question', 'is_impossible', 'id', 'answers'],
    num_rows: 1676
})

In [117]:
test = s['train'].select(range(1676, len(s['train'])))
test['document_id'][-1]

1713

In [125]:
set(train['document_id']).intersection(set(test['document_id']))

set()

In [157]:
train.save_to_disk('train')

Saving the dataset (0/1 shards):   0%|          | 0/1676 [00:00<?, ? examples/s]

In [158]:
from datasets import load_from_disk
reloaded_dataset = load_from_disk("train")

In [161]:
from datasets import DatasetDict

reloaded_dataset = ({"train": reloaded_dataset})


In [164]:
reloaded_dataset

{'train': Dataset({
     features: ['document_id', 'context', 'question', 'is_impossible', 'id', 'answers'],
     num_rows: 1676
 })}

In [167]:
all_questions = reloaded_dataset['train']['question']

In [175]:
import pickle5 as pickle
with open('../../data/COVID-QA/ents_spacy.pkl', 'rb') as f:
    ents_main = pickle.load(f)[:10]

In [176]:
ents_main

['inhibitors',
 'LPS-primed',
 'Transparent',
 'HRP-labeled goat-anti-human IgG',
 'pathogenic mechanisms',
 'oro-fecal route',
 'gemcitabine lack',
 'Insectivorous bats',
 'acute respiratory viral infections',
 'biological process']

In [173]:
train_dataset = load_dataset("parquet", data_files='../../data/COVID-QA/generated_corpus_47k/agg_gens.parquet')

Found cached dataset parquet (/home/saptarshi/.cache/huggingface/datasets/parquet/default-822b55e40f9655d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [188]:
train_dataset.filter(input_columns='entity',
                    function= lambda x: x in ents_main)

Loading cached processed dataset at /home/saptarshi/.cache/huggingface/datasets/parquet/default-822b55e40f9655d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-569ac4efedbb8948.arrow


DatasetDict({
    train: Dataset({
        features: ['entity', 'prompt', 'context', '__index_level_0__'],
        num_rows: 10
    })
})